In [ ]:
! pip install Wikipedia-API

In [ ]:
import wikipediaapi
import re
import pandas as pd
from tqdm import tqdm
import json
import os 
import time 

"""DB & progress paths (better for the files to be stored in the drive
 in case you're using a notebook on the cloud for limited sessions)"""

pathbd= "/content/drive/MyDrive/DB" # Change this
pathp = "/content/drive/MyDrive/ProgressScraping"

titles = pd.read_csv(r'/content/drive/MyDrive/ProgressScraping/titles.csv', sep='\t')


article_titles = titles.drop_duplicates().reset_index(drop=True)

# Check if progress file exists
progress_file = os.path.join(pathp,"scraping_progress.json")
progress = {}
try:
    with open(progress_file, 'r') as file:
        progress = json.load(file)
except FileNotFoundError:
    pass

#depening on what you're looking for you may  want to not remove this

def clean_text(text):
    # Remove links
    text = re.sub(r'\[.*?\]', '', text)
    
    # Remove special characters or unwanted symbols
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Trim leading and trailing whitespace
    text = text.strip()
    
    return text

# Function to scrape the content of a Wikipedia article

wiki_wiki = wikipediaapi.Wikipedia(
        language='', # language
        extract_format=wikipediaapi.ExtractFormat.WIKI
)
def scrape_wikipedia_article(article_title):

    p_wiki = wiki_wiki.page(article_title)
    if p_wiki.exists():
      return p_wiki.text
    else :
      return False

# Scrape the content for each article title and save as text files
# Scrape the content for each article title
pbar = tqdm(article_titles)
for title in pbar:
    if title in progress:
        # Skip already scraped articles
        pbar.set_postfix({'status': 'Skipped'})
        continue
    
    # Scrape the article
    time.sleep(0.5)
    try :
      article_content = scrape_wikipedia_article(title)
    except:
      continue
    
    if article_content:
        # Save the content in a text file
        filename = f"{title.replace('/','_')}.txt"
        filename = os.path.join(pathbd,filename)
        with open(filename, "w", encoding="utf-8") as file:
            file.write(clean_text(article_content))
        
        # Update the progress
        progress[title] = True
        with open(progress_file, 'w') as file:
            json.dump(progress, file)
        
        pbar.set_postfix({'status': 'Scraped'})
        print(f"Content saved for article: {title}")
    else:
        pbar.set_postfix({'status': 'Failed'})
        print(f"Failed to scrape content for article: {title}")

pbar.close()


In [ ]:
"""Some help with the post processing of the text"""
# Set the directory path in Google Drive
directory_path = '/content/drive/MyDrive/DB'

# List all the files in the directory
files = os.listdir(directory_path)

# Specify the strings to search for in the filenames
strings_to_search = ['something'] #some ids to removes like "user:" '

# Iterate over the files and remove the ones matching the strings
for file_name in files:
    for string in strings_to_search:
        if string in file_name:
            file_path = os.path.join(directory_path, file_name)
            os.remove(file_path)
            print(f"Removed file: {file_name}")
            break

In [ ]:
"""filtering the crap"""

folder_path = "/content/drive/MyDrive/DB

# Create a dictionary to store the filenames without duplicates

unique_files = {}

# Function to normalize the filename by removing variations
def normalize_filename(filename):
    # Remove variations in the use of (1) or underscore
    normalized_name = re.sub(r'\(\d+\)|_', '', filename)

    # Replace variations in letter style (ṛ vs. r, ǧ vs. j, a vs. e)
    normalized_name = normalized_name.replace('ṛ', 'r').replace('ǧ', 'j').replace('e', 'a')

    return normalized_name

# Iterate through each file in the folder
for filename in os.listdir(folder_path):
    # Extract the base filename by removing leading/trailing spaces and reducing multiple spaces or underscores
    base_name = re.sub(r'\s+|_', ' ', filename.strip())

    # Normalize the base name
    normalized_base_name = normalize_filename(base_name)

    # Check if the normalized base name already exists in the dictionary
    if normalized_base_name in unique_files:
        existing_file = unique_files[normalized_base_name]

        # Compare the file name lengths to keep the shorter one
        if len(filename) < len(existing_file):
            unique_files[normalized_base_name] = filename
    else:
        unique_files[normalized_base_name] = filename

# Iterate through the dictionary and remove duplicate files
for normalized_base_name, filename in unique_files.items():
    file_path = os.path.join(folder_path, filename)
    print(f"Keeping file: {file_path}")

    # Remove the duplicates
    for other_filename in os.listdir(folder_path):
        if other_filename != filename:
            normalized_other_base_name = normalize_filename(re.sub(r'\s+|_', ' ', other_filename.strip()))
            if normalized_other_base_name == normalized_base_name:
                other_file_path = os.path.join(folder_path, other_filename)
                os.remove(other_file_path)
                print(f"Removed duplicate file: {other_file_path}")
